## Imports

In [58]:
import copy
import gurobipy as gp
import json
import numpy as np
import pandas as pd
import pprint
from gurobipy import GRB

pp = pprint.PrettyPrinter(indent=4)

## Read Data

In [59]:
arrivals_df = pd.read_excel("Arrivals.xlsx")
arrivals_df["Type"] = "Arrival"

departures_df = pd.read_excel("Departures.xlsx")
departures_df["Type"] = "Departure"

## Save Data as CSV

In [60]:
arrivals_df.to_csv("data/arrivals.csv")
departures_df.to_csv("data/departures.csv")

## Create Transfer Passengers

In [ ]:
arrivals_df["final_passengers"] = copy.copy(arrivals_df["Passengers"])
departures_df["final_passengers"] = copy.copy(departures_df["Passengers"])

In [44]:
TRANSFER_PERCENTAGE = 0.25

In [45]:
np.random.seed(100)

In [46]:
arrivals_df

,Flight Number,Arrival Time,Departure Time,Passengers,Type,final_passengers
0,2,25,136,106,Arrival,106
1,3,41,161,147,Arrival,147
2,6,73,187,187,Arrival,187
3,7,77,188,121,Arrival,121
4,10,111,233,94,Arrival,94
...,...,...,...,...,...,...
63,120,1325,1447,127,Arrival,127
64,124,1369,1499,88,Arrival,88
65,126,1390,1513,240,Arrival,240
66,127,1398,1513,87,Arrival,87


In [47]:
P = dict()

In [48]:
np.random.seed(100)
for i, arrival_row in arrivals_df.iterrows():
    flight_i = str(arrival_row["Flight Number"])
    flight_i_arrival_time = arrival_row["Arrival Time"]
    
    for j, departure_row in departures_df.iterrows():
        flight_j = str(departure_row["Flight Number"])
        flight_j_departure_time = departure_row["Departure Time"]
        
        if (flight_i_arrival_time + 40 < flight_j_departure_time and 
            flight_j_departure_time - flight_i_arrival_time <= 480):
            y = np.random.randint(0, 15)
            
            flight_i_num_passengers_initial = arrivals_df.loc[i, "Passengers"]
            flight_i_num_passengers_current = arrivals_df.loc[i, "final_passengers"]
            flight_j_num_passengers_initial = departures_df.loc[j, "Passengers"]
            flight_j_num_passengers_current = departures_df.loc[j, "final_passengers"]
            
            if ((flight_i_num_passengers_current - y >= (1 - TRANSFER_PERCENTAGE) * flight_i_num_passengers_initial) and
                (flight_j_num_passengers_current - y >= (1 - TRANSFER_PERCENTAGE) * flight_j_num_passengers_initial) and
                (y > 0)):
                arrivals_df.loc[i, "final_passengers"] -= y
                departures_df.loc[j, "final_passengers"] -= y
                if flight_i not in P:
                    P[flight_i] = dict()
                P[flight_i][flight_j] = y

In [49]:
pp.pprint(P)

{   '10': {'12': 1, '4': 8, '9': 14},
    '101': {'119': 10, '122': 11, '123': 3, '125': 10, '128': 4},
    '102': {'119': 6, '121': 1, '122': 8, '123': 7, '125': 1, '128': 12},
    '103': {'107': 3, '119': 6, '122': 9, '123': 14, '125': 6, '128': 6},
    '105': {'107': 2, '109': 1, '125': 1, '128': 10, '130': 6},
    '106': {'123': 4, '125': 5, '130': 8},
    '108': {'113': 2, '121': 2, '128': 1, '130': 9},
    '11': {   '12': 10,
              '17': 7,
              '19': 6,
              '20': 12,
              '21': 3,
              '26': 4,
              '4': 3,
              '5': 6},
    '110': {'113': 1, '123': 1, '128': 5},
    '114': {'119': 3, '125': 4},
    '118': {'122': 1, '123': 3},
    '120': {'123': 4, '130': 3},
    '13': {'12': 2, '17': 13, '19': 7},
    '14': {   '12': 3,
              '17': 7,
              '19': 13,
              '20': 9,
              '23': 5,
              '26': 9,
              '27': 6,
              '28': 6},
    '15': {'12': 9, '17': 2, '19': 

In [50]:
arrivals_df["percentage"] = arrivals_df["final_passengers"] / arrivals_df["Passengers"]
print(min(arrivals_df["percentage"]))
arrivals_df

0.75


,Flight Number,Arrival Time,Departure Time,Passengers,Type,final_passengers,percentage
0,2,25,136,106,Arrival,80,0.754717
1,3,41,161,147,Arrival,111,0.755102
2,6,73,187,187,Arrival,143,0.764706
3,7,77,188,121,Arrival,91,0.752066
4,10,111,233,94,Arrival,71,0.755319
...,...,...,...,...,...,...,...
63,120,1325,1447,127,Arrival,120,0.944882
64,124,1369,1499,88,Arrival,88,1.000000
65,126,1390,1513,240,Arrival,240,1.000000
66,127,1398,1513,87,Arrival,87,1.000000


In [51]:
departures_df["percentage"] = departures_df["final_passengers"] / departures_df["Passengers"]
print(min(departures_df["percentage"]))
departures_df

0.75


,Flight Number,Arrival Time,Departure Time,Passengers,Type,final_passengers,percentage
0,1,11,126,208,Departure,186,0.894231
1,4,47,177,141,Departure,109,0.773050
2,5,59,179,143,Departure,112,0.783217
3,8,95,219,123,Departure,93,0.756098
4,9,99,214,124,Departure,93,0.750000
...,...,...,...,...,...,...,...
57,122,1343,1466,239,Departure,180,0.753138
58,123,1353,1482,164,Departure,123,0.750000
59,125,1378,1489,137,Departure,104,0.759124
60,128,1408,1530,172,Departure,131,0.761628


## Save Outputs

In [52]:
arrivals_df.to_csv(f"data/transfers/{TRANSFER_PERCENTAGE}/arrivals.csv")
departures_df.to_csv(f"data/transfers/{TRANSFER_PERCENTAGE}/departures.csv")

In [53]:
outpath = f"data/transfers/{TRANSFER_PERCENTAGE}/transfers.json"
with open(outpath, "w") as outfile:
    json.dump(P, outfile)

In [54]:
with open(outpath) as json_file:
    P2 = json.load(json_file)
assert(P == P2)

In [55]:
P2

{'2': {'1': 8, '4': 8, '5': 3, '8': 7},
 '3': {'1': 8, '4': 4, '5': 11, '8': 12, '28': 1},
 '6': {'1': 4,
  '4': 4,
  '5': 11,
  '8': 11,
  '9': 3,
  '12': 7,
  '17': 1,
  '19': 1,
  '28': 2},
 '7': {'1': 2, '4': 5, '9': 14, '12': 1, '20': 7, '38': 1},
 '10': {'4': 8, '9': 14, '12': 1},
 '11': {'4': 3,
  '5': 6,
  '12': 10,
  '17': 7,
  '19': 6,
  '20': 12,
  '21': 3,
  '26': 4},
 '13': {'12': 2, '17': 13, '19': 7},
 '14': {'12': 3,
  '17': 7,
  '19': 13,
  '20': 9,
  '23': 5,
  '26': 9,
  '27': 6,
  '28': 6},
 '15': {'12': 9, '17': 2, '19': 3, '21': 7, '22': 1, '26': 5},
 '16': {'12': 4, '21': 8, '22': 14, '26': 5, '27': 2},
 '18': {'12': 6,
  '19': 2,
  '21': 5,
  '22': 3,
  '26': 2,
  '27': 3,
  '40': 1,
  '47': 1},
 '24': {'21': 8,
  '23': 10,
  '26': 12,
  '27': 2,
  '28': 2,
  '31': 7,
  '36': 2,
  '38': 1,
  '40': 2},
 '25': {'21': 14,
  '22': 3,
  '23': 12,
  '26': 5,
  '27': 2,
  '28': 6,
  '31': 1,
  '35': 10,
  '38': 1,
  '45': 2},
 '29': {'23': 6,
  '26': 4,
  '27': 6,
  '2

## Testing

In [26]:
from pandas import DataFrame

pi = 0.25
Arrival_shape = arrivals_df.shape[0]
Departure_shape = departures_df.shape[0]
ind = np.arange(Departure_shape) + Arrival_shape
P = DataFrame(0, index= np.arange(Arrival_shape), columns = (ind))
np.random.seed(100)
for i in np.arange(Arrival_shape):
    initial = arrivals_df['Passengers'][i]
    for j in np.arange(Departure_shape):
        if (arrivals_df['Arrival Time'][i] + 40 < departures_df['Departure Time'].iloc[j] and 
            departures_df['Departure Time'].iloc[j] - arrivals_df['Arrival Time'][i] <= 480):
            x = arrivals_df['Passengers'][i]
            y = np.random.randint(0,15)
            if x - y >= (1 - pi) * initial:
                P.iloc[[i],[j]] = y
                arrivals_df['Passengers'][i] = x - y

<ipython-input-26-0deb028598be>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrivals_df['Passengers'][i] = x - y


In [27]:
P

,68,69,70,71,72,73,74,75,76,77,...,120,121,122,123,124,125,126,127,128,129
0,8,8,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,4,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,4,11,11,3,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,5,11,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,8,0,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0,0,0,0,0,0,0,0,0,0,...,0,14,4,6,0,0,0,0,0,0
64,0,0,0,0,0,0,0,0,0,0,...,0,0,5,4,1,6,0,0,0,0
65,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,12,13,8,5,0,0
66,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,9,0,0,6,0,0
